# 02 — Data Understanding (CRISP-DM)

Nesta etapa do CRISP-DM, o objetivo é **entender profundamente os dados disponíveis**,
avaliando sua estrutura, qualidade, volume e possíveis limitações.

Não há, neste momento, qualquer tipo de tratamento ou limpeza dos dados.
O foco é responder perguntas como:
- Quais dados estão disponíveis?
- Como eles estão estruturados?
- Existem valores ausentes?
- Há dados sensíveis?
- O que pode impactar as análises futuras?

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [ ]:
from pathlib import Path

DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

len(csv_files), csv_files

In [ ]:
dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

df_raw.shape

In [ ]:
df_raw.head()

In [ ]:
df_raw.tail()

## Observações sobre codificação dos arquivos

Durante a ingestão dos dados, foi identificado que os arquivos CSV
não utilizam codificação UTF-8, resultando em erros de leitura.

Após análise, foi adotada a codificação `latin1`, comum em dados
oriundos de sistemas legados da administração pública brasileira,
permitindo a correta leitura de caracteres acentuados.


In [ ]:
schemas = {
    file.name: set(pd.read_csv(file, encoding="latin1", sep=";", nrows=0).columns)
    for file in csv_files
}

schemas

In [ ]:
unique_schemas = set(tuple(sorted(cols)) for cols in schemas.values())
len(unique_schemas)

In [ ]:
path_raw = Path("../data/raw")

dfs = []

for file in path_raw.glob("*.csv*"):
    df = pd.read_csv(file, sep=";", encoding="latin1")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

df_raw.shape

In [ ]:
df_raw.columns

In [ ]:
# Contar servidores únicos
num_servidores = df_raw["Nome"].nunique()

print(f"Total de servidores únicos no ano: {num_servidores}")

In [ ]:
servidores_unicos = df_raw["Nome"].drop_duplicates().sort_values()
servidores_unicos.head(5)

In [ ]:
# Separar tipo de pagamento e mês
df_raw[["tipo_pagamento", "mes_nome"]] = df_raw["Referência"].str.split(" - ", expand=True)

# Mapear mês para número
mapa_meses = {
    "Janeiro": 1,
    "Fevereiro": 2,
    "Março": 3,
    "Abril": 4,
    "Maio": 5,
    "Junho": 6,
    "Julho": 7,
    "Agosto": 8,
    "Setembro": 9,
    "Outubro": 10,
    "Novembro": 11,
    "Dezembro": 12
}

df_raw["mes_num"] = df_raw["mes_nome"].map(mapa_meses)

# Conferir resultado
df_raw[["Referência", "tipo_pagamento", "mes_nome", "mes_num"]].head(5)

In [ ]:
df_raw = df_raw[df_raw["Referência"].notna()]

## Resumo da compreensão dos dados

- Os dados estão distribuídos em múltiplos arquivos CSV, um por período.
- Todos os arquivos apresentam **schema consistente**, permitindo concatenação segura.
- A codificação utilizada é `latin1`, compatível com sistemas legados.
- Não foram identificadas divergências estruturais entre os meses.
- Existem colunas com dados potencialmente sensíveis, que serão tratadas
  nas próximas etapas.
